In [2]:
import pandas as pd
import numpy as np

In [8]:
# Array of all years in dataset
years = ['2014','2015','2016','2017','2018','2019','2020','2021','2022']

# Store column names present in data
columns = pd.read_csv(f'London_2022_elite_results.csv').columns.tolist() # 2022 arbitrarily selected
columns = columns + ['Year']

# Initialize new dataframes
elite_df = pd.DataFrame(columns=columns)
mass_df = pd.DataFrame(columns=columns)

# Stack elite dataframes on year
for year in years:
    df = pd.read_csv(f'London_{year}_elite_results.csv')
    df['Year'] = year
    elite_df = pd.concat([elite_df, df], axis=0, ignore_index=True)

# Remove 2020 because there was only an elite marathon that year
years = ['2014','2015','2016','2017','2018','2019','2021','2022']

# Stack mass dataframes on year
for year in years:
    df = pd.read_csv(f'London_{year}_mass_results.csv')
    df['Year'] = year
    mass_df = pd.concat([mass_df, df], axis=0, ignore_index=True)

In [9]:
def SeparateName(df):
    """Separate first name, last name, and country code that are grouped in original data"""

    # Iterate through the DataFrame and split strings based on conditions
    rows_to_remove = []  # Create a list to store row indices to be removed

    for index, row in df.iterrows():
        input_string = row['Name']
        split_parts = input_string.split(' (')
        
        try:
            last_name, rest = split_parts[0].split(', ')
            first_name = rest

            # Check if there are enough elements in split_parts
            if len(split_parts) > 1:
                country = split_parts[1][:-1]
            else:
                country = None
                rows_to_remove.append(index)

            # Update the DataFrame
            df.at[index, 'Last Name'] = last_name.lower()  # Cast to lowercase due to reporting inconsistencies
            df.at[index, 'First Name'] = first_name.lower()
            df.at[index, 'Country'] = country
        except (ValueError, IndexError):
            rows_to_remove.append(index)

    # Drop the original 'Name' column if you want to remove it
    df = df.drop(columns=['Name'])

    # Remove rows with invalid data
    df = df.drop(rows_to_remove)

    return df



# Call the above function on both dataframes
elite_df = SeparateName(elite_df)
mass_df = SeparateName(mass_df)

In [25]:
# Convert finish time to seconds for ease of calculations
def time_to_seconds(time_str):
    try:
        return pd.to_timedelta(time_str).total_seconds()
    except ValueError:
        return None
    
elite_df['Total Seconds'] = elite_df['Finish Time'].apply(time_to_seconds)
elite_df['Half Seconds'] = elite_df['Half Time'].apply(time_to_seconds)
elite_df['Gender'] = elite_df['Gender'].replace('W', 'F')
mass_df['Total Seconds'] = mass_df['Finish Time'].apply(time_to_seconds)

In [27]:
### Import 2023 data from excel file and convert times to seconds

def time_to_seconds_2023(time_str):
    try:
        return pd.to_timedelta(time_str).total_seconds()
    except ValueError:
        return time_str.hour * 3600 + time_str.minute * 60 + time_str.second
    
    
men_2023 = pd.read_excel('London_2023_elite_results.xlsx',sheet_name='Men')
women_2023 = pd.read_excel('London_2023_elite_results.xlsx',sheet_name='Women')
df_2023 = pd.concat([men_2023,women_2023], axis=0, ignore_index=True)
df_2023['Total Seconds'] = df_2023['Finish Time'].apply(time_to_seconds_2023)
df_2023['Half Seconds'] = df_2023['Half Time'].apply(time_to_seconds_2023)

In [28]:
### Combine 2023 data with older data
elite_df = pd.concat([elite_df,df_2023],axis=0,ignore_index=True)
print(elite_df)

     Overall Place  Gender Place  Category Place       Club Runner Number  \
0              1.0           1.0             1.0        NaN             2   
1              2.0           2.0             2.0        NaN             8   
2              3.0           3.0             3.0        NaN             1   
3              4.0           4.0             4.0        NaN             5   
4              5.0           5.0             5.0        NaN             6   
..             ...           ...             ...        ...           ...   
391           32.0           9.0             9.0        NaN            81   
392           33.0          10.0            10.0        NaN            71   
393           35.0          11.0            11.0  Charnwood            75   
394           36.0          12.0            12.0        NaN            79   
395           37.0          13.0            13.0        NaN            72   

    Gender Category  Event Half Time Finish Time  Year  Last Name First Nam

In [33]:
# Combine dataframes
both_df = pd.concat([elite_df, mass_df], axis=0, ignore_index=True)

# Save all three dataframes as csvs
# both_df.to_csv('London_Marathon_2014_to_2022.csv', index=False) 
# elite_df.to_csv('London_Marathon_Elite_2014_to_2023.csv', index=False)
# mass_df.to_csv('London_Marathon_Mass_2014_to_2022.csv', index=False)

In [44]:
### Calculate the size ratio between elite and total
elite_men = elite_df[elite_df['Gender'] == 'M']
print(elite_men['Year'].value_counts())
elite_women = elite_df[elite_df['Gender'] == 'F']
print(elite_women['Year'].value_counts())

both_men = both_df[both_df['Gender'] == 'M']
print(both_men['Year'].value_counts())
both_women = both_df[(both_df['Gender'] == 'F') | (both_df['Gender'] == 'W')]
print(both_women['Year'].value_counts())

men_ratio = elite_men['Year'].value_counts()/both_men['Year'].value_counts()
women_ratio = elite_women['Year'].value_counts()/both_women['Year'].value_counts()

men_ratio = men_ratio[men_ratio != 1]
women_ratio = women_ratio[women_ratio != 1]

print(men_ratio.mean())
print(women_ratio.mean())

2016    25
2017    25
2019    25
2020    25
2023    24
2015    21
2022    21
2021    20
2014    18
2018    18
Name: Year, dtype: int64
2017    25
2019    21
2015    20
2016    19
2021    19
2020    18
2014    14
2018    13
2023    13
2022    12
Name: Year, dtype: int64
2019    24803
2016    23935
2017    23841
2022    23721
2018    23695
2015    23242
2014    22626
2021    21356
2020       25
2023       24
Name: Year, dtype: int64
2019    17801
2022    16889
2018    16427
2017    15485
2016    15044
2015    14393
2021    14376
2014    13283
2020       18
2023       13
Name: Year, dtype: int64
0.0009226979863436288
0.0011655285765913798


NameError: name 'recent_df' is not defined